## Importing libreries

---

In [1]:
# Data treatment
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Path
import sys
sys.path.append('../')

# Config
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)

from src.support import find_outliers_iqr

import warnings
warnings.filterwarnings('ignore')

## Load data

---

In [2]:
path = "../data/output/data_clean.csv"

df = pd.read_csv(path)
# Conveting dates to datetime
df['Posting Date'] = pd.to_datetime(df['Posting Date'], dayfirst=True)

#### Currency

Although data does not explicitly mention the currency, since these are Brazilian government data, it is reasonable to assume that the amounts are in Brazilian Reais (BRL), as this is the official currency of Brazil. This would be consistent with the standard practices for publishing financial data by the government of that country.

## Revenue Distribution by Economic Category

---

Let's start analyzing the most significant revenue categories and their contribution to total revenues. We can check quickly the different types of Economic Categories.

In [3]:
df['Economic Category'].unique()

array(['Current Revenues', 'Capital Revenues', 'No Information',
       'Intra-Budgetary Current Revenues',
       'Intra-Budgetary Capital Revenues'], dtype=object)

Show the number of operations with an Actual Amount registered grouped by Economic Category

In [4]:
df.groupby('Economic Category')['Actual Amount'].count().round(0)

Economic Category
Capital Revenues                     27065
Current Revenues                    906945
Intra-Budgetary Capital Revenues        86
Intra-Budgetary Current Revenues     14566
No Information                       17893
Name: Actual Amount, dtype: int64

With this information, we can show some initial insights about the data:

1. `Current Revenues` is the most represented category with 906945 entries. `Current Revenues` probably include taxes, fees, and other regular government income.

2. `Intra-Budgetary Current Revenues` has far fewer entries, with a total of 14567. This difference indicates that intra-budgetary revenues, those generated within the government's own budget, represent a much smaller portion of total entries.

3. `Capital Revenues` has 27065 entries, indicating that capital revenues (which might include revenue from asset sales, investments, or financing) are also important but not as common as current revenues. However, this kind of transactions are usually way more expensive than `Current Revenues` as we will see in the next step.

4. `Intra-Budgetary Capital Revenues` with only 86 entries, shows that intra-budgetary capital revenues are unusual. This could imply that there are few events or sources of capital income occurring within the government's own budget.

5. `No Information` has 17893 entries. Depending on how big the mean amount is it could be a big loss of information or couldn't be.

Let's now move to the value of operations with an Actual Amount registered grouped by Economic Category

In [5]:
df.groupby('Economic Category')['Actual Amount'].sum().round(0)

Economic Category
Capital Revenues                    1.200415e+13
Current Revenues                    1.202508e+13
Intra-Budgetary Capital Revenues    2.141127e+10
Intra-Budgetary Current Revenues    2.805787e+11
No Information                      3.271665e+11
Name: Actual Amount, dtype: float64

We can extract again some valuable insights:

1. `Current Revenues`:
   - The total actual revenues amount to 1.202 x 10¹³ (approximately 12.02 trillion Brazilian reais). This confirms that current revenues are a huge source of income for the government, as already suggested by the number of entries in this category.

2. `Intra-Budgetary Current Revenues`:
   - The total actual revenues amount to 2.8058 x 10¹¹ (280.58 billion reais). Although much smaller than `Current Revenues`, it is still a significant amount.

3. `Capital Revenues`:
   - The total actual revenues amount to 1.20 x 10¹³ (approximately 12 trillion reais). Capital revenues are almost equivalent in magnitude to current revenues, indicating that this category also plays a crucial role in government revenue.

4. `Intra-Budgetary Capital Revenues`:
   - The total actual amount is 2.141 x 10¹⁰ (21.41 billion reais). These revenues are a small fraction compared to the total capital revenues.

5. `No Information`:
   - The total amount is 3.27166 x 10¹¹ (327.17 billion reais). Represents a 1.33% out of the total with unknown category. Despite seeming to be low it's a huge amount that should have been labeled.

---

Now we can compute the average difference between projected and actual revenues for each category. This only makes sense for entries that have both numbers available, which is only 8837 entries

In [6]:
df_diff = df[df['Actual Amount'].notna() & df['Updated Budgeted Amount'].notna()][['Economic Category', 'Actual Amount', 'Updated Budgeted Amount']]
df_diff

,Economic Category,Actual Amount,Updated Budgeted Amount
20,Capital Revenues,985939.65,640279.0
21,Capital Revenues,194307.50,358413.0
22,No Information,68323.99,111097.0
23,Current Revenues,107548.62,350042.0
25,Current Revenues,5145837.84,4639104.0
...,...,...,...
1024565,Current Revenues,1098.41,907063.0
1025301,Current Revenues,480.00,517775.0
1025530,Current Revenues,1483.23,1323033.0
1025680,Current Revenues,135.90,407371.0


In [7]:
df_diff['Difference'] = df_diff['Actual Amount'] - df_diff['Updated Budgeted Amount']
df_diff['Porcentual_difference'] = df_diff['Difference'] / df_diff['Updated Budgeted Amount'] * 100

df_diff.groupby('Economic Category')['Difference'].mean().round(0)

Economic Category
Capital Revenues                   -2.129977e+10
Current Revenues                   -4.509515e+08
Intra-Budgetary Current Revenues   -1.193434e+08
No Information                     -4.687883e+09
Name: Difference, dtype: float64

It makes more sense to show the porcentual difference:

In [8]:
df_diff.groupby('Economic Category')['Porcentual_difference'].agg(['mean', 'median'])

,mean,median
Economic Category,,
Capital Revenues,2785.083178,-46.251895
Current Revenues,9900.145758,-48.355853
Intra-Budgetary Current Revenues,199.616772,-38.589202
No Information,1613.218577,-31.114211


We can see that mean and median are extremely different. This is usually caused by outliers, let's check

In [9]:
# Number of outliers
find_outliers_iqr(df_diff, 'Porcentual_difference').sort_values(by = 'Porcentual_difference').shape[0]

805

There are 805 outliers, but some of them are very extreme:

In [10]:
# Number of outliers using 3000 times the IQR instead of 1.5 (they exist)
outliers = find_outliers_iqr(df_diff, 'Porcentual_difference', width = 3000).sort_values(by = 'Porcentual_difference', ascending=False).index.tolist()

len(outliers)

7

There are a lot of outliers and some of them extremely far from the median

In [11]:
for i in outliers:

    print(df.iloc[i,11])

NaT
2014-12-31 00:00:00
2013-12-31 00:00:00
2015-12-31 00:00:00
2014-12-31 00:00:00
2013-12-31 00:00:00
2013-12-31 00:00:00


The entries being posted on December 31st likely reflect year-end financial adjustments, where revenues accumulated throughout the year are recognized in one batch. This could result from accounting practices like delayed reporting or systematic year-end postings.

For other dates, there may be other causes like underestimated revenues, unexpected demand for services, large projects/contracts not initially accounted for in the budget or even data entry errors or misclassification.

---

### Temporal analysis

In [12]:
# Now, let's evaluate the temporal trends by calculating the sum of actual income per year.
df.groupby('Fiscal Year')['Actual Amount'].sum()

Fiscal Year
2013    1.663430e+12
2014    2.211514e+12
2015    2.634219e+12
2016    2.787181e+12
2017    2.476073e+12
2018    2.865977e+12
2019    2.880424e+12
2020    3.460501e+12
2021    3.679059e+12
Name: Actual Amount, dtype: float64

We can see a increasement of the incomes over time except for a slight decreasement in 2016-2017 that can be related to Brasil economical crisis in 2014-2017

In [13]:
# Now, let's evaluate the temporal trends by calculating the sum of actual income per year and month.
for i in range(2013, 2022):
    
    print(i, df[df['Fiscal Year'] == i].groupby(df['Posting Date'].dt.month)['Actual Amount'].sum())

2013 Posting Date
12.0    1.617267e+12
Name: Actual Amount, dtype: float64
2014 Posting Date
12.0    2.192906e+12
Name: Actual Amount, dtype: float64
2015 Posting Date
12.0    2.630488e+12
Name: Actual Amount, dtype: float64
2016 Posting Date
1.0     2.445406e+11
2.0     1.443923e+11
3.0     3.234955e+11
4.0     2.215033e+11
5.0     1.662483e+11
6.0     1.752807e+11
7.0     2.347508e+11
8.0     1.788412e+11
9.0     1.913556e+11
10.0    2.877622e+11
11.0    1.512721e+11
12.0    4.413237e+11
Name: Actual Amount, dtype: float64
2017 Posting Date
1.0     3.554494e+11
2.0     1.604895e+11
3.0     1.949693e+11
4.0     1.909670e+11
5.0     1.517739e+11
6.0     1.733170e+11
7.0     1.801296e+11
8.0     2.202914e+11
9.0     2.212834e+11
10.0    2.109477e+11
11.0    1.686580e+11
12.0    1.703508e+11
Name: Actual Amount, dtype: float64
2018 Posting Date
1.0     3.058722e+11
2.0     1.599130e+11
3.0     2.478358e+11
4.0     2.647583e+11
5.0     1.734651e+11
6.0     1.754082e+11
7.0     2.402284e+1

1. **2013-2015**: The data only includes the total amount for December, and these amounts are significantly higher compared to individual months in subsequent years. This suggests that income may have been reported in a lump sum at the end of the year during these early years.

2. **2016 onwards**: The income is reported monthly, providing a clearer picture of income flows. Notable trends include:
   - **2016**: A peak in December with an actual amount of approximately 4.41 x 10¹¹.

   - **2017-2019**: Monthly income appears fairly consistent.

   - **2020**: Significant spikes in August (6.22 x 10¹¹) and December, possibly reflecting special circumstances (maybe COVID-19-related budget adjustments).

   - **2021**: Again spikes in April and September but quite homogeneous across the year.

---

We can also have some quick insights of `Actual Amount` grouped by `Superior Agency`, `Agency` and `Managing Unit`

In [14]:
df.groupby('Superior Agency')['Actual Amount'].describe().T

Superior Agency,Advocacia-Geral da União,Controladoria-Geral da União,"Ministério da Agricultura, Pecuária e Abastec",Ministério da Cidadania,"Ministério da Ciência, Tecnologia, Inovações",Ministério da Defesa,Ministério da Economia,Ministério da Educação,Ministério da Infraestrutura,Ministério da Justiça e Segurança Pública,"Ministério da Mulher, Família e Direitos Huma",Ministério da Pesca e Aquicultura,Ministério da Previdência Social,Ministério da Saúde,Ministério das Comunicações,"Ministério das Mulheres, Igualdade Racial, da",Ministério das Relações Exteriores,Ministério de Minas e Energia,Ministério do Desenvolvimento Agrário,Ministério do Desenvolvimento Regional,Ministério do Esporte,Ministério do Meio Ambiente,Ministério do Trabalho e Emprego,Ministério do Turismo,Presidência da República
count,3.732000e+03,2.340000e+02,4.337900e+04,5.489000e+03,3.102600e+04,9.021200e+04,1.885970e+05,3.385350e+05,3.554500e+04,3.773800e+04,1.065000e+03,3.510000e+02,9.700000e+01,3.117900e+04,2.785600e+04,2.480000e+02,3.365000e+03,3.104200e+04,6.290000e+02,2.989100e+04,1.340000e+02,2.555500e+04,6.400000e+02,2.548700e+04,4.648000e+03
mean,7.509472e+04,3.172387e+04,1.255170e+06,6.590077e+05,1.089814e+06,1.089369e+06,1.238519e+08,6.385683e+05,3.655761e+06,6.439728e+05,1.627008e+05,1.593145e+05,1.396442e+05,1.176563e+06,1.285614e+06,5.501818e+05,6.824171e+05,6.084961e+06,8.066418e+05,6.199545e+05,4.012129e+05,2.570217e+05,2.106916e+07,5.169181e+05,3.330343e+06
std,1.496635e+06,1.252237e+05,2.695636e+07,1.625517e+07,2.900641e+07,2.055599e+07,3.224854e+09,5.805303e+07,5.386872e+07,1.000687e+07,9.239152e+05,8.881368e+05,9.857254e+05,3.684991e+07,3.423583e+07,2.155432e+06,6.078540e+06,3.039367e+08,5.540407e+06,7.055228e+06,1.487397e+06,3.013930e+06,2.816869e+08,1.532192e+07,2.083724e+07
min,-2.692618e+07,2.000000e+00,-5.000000e+07,-9.295862e+08,-4.288879e+08,-1.526649e+08,-1.562859e+11,-1.160700e+08,-1.280973e+08,-7.243899e+08,-1.903351e+04,-7.839058e+04,-1.518457e+05,-7.594131e+07,-1.069885e+08,-7.153700e+02,-9.384494e+06,-1.175745e+10,-1.181910e+06,-5.315246e+07,-2.850535e+05,-2.746580e+07,-9.639166e+05,-1.296744e+08,-7.669350e+05
25%,1.638625e+02,1.678500e+02,5.190550e+02,2.926300e+02,4.800000e+02,3.260000e+02,1.402420e+03,1.526500e+02,1.347070e+03,6.768050e+02,6.335000e+02,2.233990e+03,4.450400e+02,7.233700e+02,3.584550e+02,2.612950e+02,4.489000e+01,2.599202e+03,1.434230e+03,2.969150e+02,9.900546e+04,3.709650e+02,3.700325e+02,2.704500e+02,4.272148e+03
50%,1.626260e+03,1.131775e+03,8.944710e+03,1.119333e+04,4.252595e+03,4.548860e+03,1.952564e+04,8.730000e+02,2.437942e+04,1.214427e+04,1.124789e+04,1.967030e+04,3.606870e+03,1.234647e+04,4.455000e+03,1.948679e+04,2.772900e+02,2.116457e+04,4.169075e+04,3.760000e+03,1.369974e+05,5.850840e+03,1.625290e+03,1.699110e+03,1.932733e+05
75%,1.035065e+04,9.887263e+03,6.334765e+04,1.070733e+05,4.779214e+04,5.711959e+04,6.050053e+05,5.473115e+03,3.821509e+05,8.765762e+04,1.356991e+05,3.180625e+04,5.808870e+03,1.551460e+05,2.914542e+04,1.500164e+05,4.081800e+03,9.310930e+04,2.361587e+05,4.554806e+04,1.659266e+05,9.162346e+04,3.512975e+03,1.289730e+04,3.636919e+05
max,6.382985e+07,1.410733e+06,3.436513e+09,4.162267e+08,3.208105e+09,2.649783e+09,7.711177e+11,1.903864e+10,5.101423e+09,9.701998e+08,1.674764e+07,8.635462e+06,9.369000e+06,3.805116e+09,2.631315e+09,1.618691e+07,2.169262e+08,3.442000e+10,9.400954e+07,4.581664e+08,1.045839e+07,1.753290e+08,4.700232e+09,1.093697e+09,8.470057e+08


In [15]:
df.groupby('Agency')['Actual Amount'].describe().T

Agency,Advocacia-Geral da União - Unidades com vínculo direto,Agência Espacial Brasileira,Agência Nacional de Aviação Civil,Agência Nacional de Energia Elétrica,Agência Nacional de Mineração,Agência Nacional de Saúde Suplementar,Agência Nacional de Transportes Aquaviários,Agência Nacional de Transportes Terrestres,Agência Nacional de Vigilância Sanitária,Agência Nacional de Águas,Agência Nacional do Cinema,"Agência Nacional do Petróleo, Gás Natural e Biocombustíveis",Agência de Desenvolvimento da Amazônia,Agêncial Nacional de Telecomunicações,Amazônia Azul Tecnologia de Defesa S.A.,Autoridade Pública Olímpica,Banco Central do Brasil - Orçamento Fiscal e Seguridade Social,Caixa de Construções de Casas para o Pessoal da Marinha do Brasil,Caixa de Financiamento Imobiliário da Aeronáutica,Casa da Moeda do Brasil,Centro Federal de Educação Tecnológica Celso Suckow da Fonseca,Centro Federal de Educação Tecnológica de Minas Gerais,Centro Nacional de Tecnologia Eletrônica Avançada S.A.,Colégio Pedro II,Comando da Aeronáutica,Comando da Marinha,Comando da Marinha - Fundo Naval,Comando do Exército,Comissão Nacional de Energia Nuclear,Comissão de Valores Mobiliários,Companhia Brasileira de Trens Urbanos,Companhia Docas do Ceará,Companhia Docas do Espírito Santo,Companhia Docas do Estado da Bahia,Companhia Docas do Estado de São Paulo,Companhia Docas do Pará,Companhia Docas do Rio Grande do Norte,Companhia Docas do Rio de Janeiro,Companhia Nacional de Abastecimento,Companhia de Desenvolvimento de Barcarena,Companhia de Desenvolvimento dos Vales do São Francisco e do Parnaíba,Companhia de Entrepostos e Armazéns Gerais de São Paulo,Companhia de Pesquisa de Recursos Minerais,Conselho Administrativo de Defesa Econômica,Conselho Nacional de Desenvolvimento Científico e Tecnológico,Controladoria-Geral da União,Controladoria-Geral da União - Unidades com vínculo direto,Departamento Nacional de Estradas de Rodagem,Departamento Nacional de Infraestrutura de Transportes,Departamento Nacional de Obras Contra as Secas,Departamento Nacional de Produção Mineral,Departamento de Polícia Federal,Departamento de Polícia Rodoviária Federal,Departamento do Fundo da Marinha Mercante,Empresa Brasil de Comunicação,Empresa Brasileira de Correios e Telégrafos,Empresa Brasileira de Pesquisa Agropecuária,Empresa Brasileira de Serviços Hospitalares,Empresa de Pesquisa Energética,Empresa de Planejamento e Logística S.A.,Empresa de Trens Urbanos de Porto Alegre,Financiadora de Estudos e Projetos,Fundação Alexandre de Gusmão,Fundação Biblioteca Nacional,Fundação Casa de Rui Barbosa,Fundação Coordenação de Aperfeiçoamento de Pessoal de Nível Superior,Fundação Cultural Palmares,Fundação Escola Nacional de Administração Pública,Fundação Instituto Brasileiro de Geografia e Estatística,Fundação Joaquim Nabuco,"Fundação Jorge Duprat Figueiredo, de Segurança e Medicina do Trabalho",Fundação Nacional de Artes,Fundação Nacional de Saúde,Fundação Nacional do Índio,Fundação Nacional do Índio - Patrimônio Indígena,Fundação Oswaldo Cruz,Fundação Osório,Fundação Universidade Federal da Grande Dourados,Fundação Universidade Federal de Ciências da Saúde de Porto Alegre,Fundação Universidade Federal de Pelotas,Fundação Universidade Federal de Rondônia,Fundação Universidade Federal de Sergipe,Fundação Universidade Federal de São Carlos,Fundação Universidade Federal de São João Del-Rei,Fundação Universidade Federal de Uberlândia,Fundação Universidade Federal do ABC,Fundação Universidade Federal do Acre,Fundação Universidade Federal do Amapá,Fundação Universidade Federal do Mato Grosso,Fundação Universidade Federal do Mato Grosso do Sul,Fundação Universidade Federal do Pampa,Fundação Universidade Federal do Piauí,Fundação Universidade Federal do Tocantins,Fundação Universidade Federal do Vale do São Francisco,Fundação Universidade de Brasília,Fundação Universidade do Amazonas,Fundação Universidade do Maranhão,Fundo Aeronáutico,Fundo Aeroviário,Fundo Constitucional de Financiamento do Cent

In [16]:
df.groupby('Managing Unit')['Actual Amount'].describe().T

Managing Unit,AG. NAC.DO PETROLEO GAS NAT.E BIOCOM,AGENCIA DE DESENVOLVIMENTO DA AMAZONIA,AGENCIA ESPACIAL BRASILEIRA/AEB,AGENCIA NACIONAL DE AVIACAO CIVIL,AGENCIA NACIONAL DE AVIACAO CIVIL - F. AEROV.,AGENCIA NACIONAL DE ENERGIA ELETRICA,AGENCIA NACIONAL DE SAUDE SUPLEMENTAR,AGENCIA NACIONAL DE TELECOMUNICACOES-SEDE,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,AGENCIA NACIONAL DE TRANSPORTES TERRESTRES,AGENCIA NACIONAL DE VIGILANCIA SANITARIA,AGENCIA NACIONAL DO CINEMA,AUTORIDADE PUBLICA OLIMPICA,BANCO CENTRAL DO BRASIL,CAIXA FINANCIAMENTO IMOBILIARIO AERONAUTICA,CAPES - DIFERENCA CAMBIAL,CASA DA MOEDA DO BRASIL,CENTRO FED.DE EDUC.TECNOL.CELSO S.DA FONSECA,CENTRO FEDERAL DE EDUCACAO TECNOLOGICA DE MG,CENTRO NACIONAL DE TECNOLOGIA ELETRONICA AVAN,CGOFC - SETORIAL/MTUR,CIA BRASILEIRA DE TRENS URBANOS-ADM.CENTRAL,CIA DE DES.DOS VALES DO S.FRANC.E DO PARNAIBA,CIA.DE ENTREPOSTOS E ARMAZENS GER.DE S.PAULO,CNEN-ORCAMENTO E FINANCAS,CNPQ - ADMINISTRACAO CENTRAL,CNPQ - DIFERENCIA CAMBIAL,COLEGIO PEDRO II,COMISSAO DE VALORES MOBILIARIOS,COMPANHIA DE DESENVOLVIMENTO DE BARCARENA,COMPANHIA DE PESQUISA DE RECURSOS MINERAIS,COMPANHIA DOCA DO RIO GRANDE DO NORTE,COMPANHIA DOCAS DO CEARA,COMPANHIA DOCAS DO ESPIRITO SANTO,COMPANHIA DOCAS DO ESTADO DA BAHIA,COMPANHIA DOCAS DO ESTADO DE SAO PAULO,COMPANHIA DOCAS DO PARA,COMPANHIA DOCAS DO RIO DE JANEIRO,COMPANHIA NACIONAL DE ABASTECIMENTO,CONSELHO ADMINISTRATIVO DE DEFESA ECONOMICA,CONSELHO NAC DE DESENV CIENT E TECNOLOGICO,"COORD-GERAL DE ORC., FIN. E CONT.-CGOFC (MTP)","COORD-GERAL DE ORCAM., FIN. E CONTABILIDADE","COORD-GERAL DE ORCAMENTO, FIN. E CONT.- CGOFC",COORD. DE PLAN. ORCAMENTO E FIN./COPLAN/JBRJ,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA,COORD.GERAL DE ORC. FIN. E CONTABILIDADE,COORD.GERAL DE ORCAM.E FIN./M.M.E,"COORD.GERAL DE ORCAMENTO, FINANCAS E CONTAB.",COORD.GERAL DE PLAN. ORC. FIN. E CONTAB./MINC,COORD.GERAL DE PLAN. ORC. FIN. E CONTAB/FNC,COORDENACAO DE ORCAMENTO E FINANCAS - COF/DPF,COORDENACAO DE ORCAMENTO E FINANCAS - PRF,COORDENACAO DE ORCAMENTO E FINANCAS DO FRGPS,COORDENACAO DE ORCAMENTO E FINANCAS.,COORDENACAO DE ORCAMENTO FINANCAS CONTABILIDA,COORDENACAO-GERAL DE ORC. E FINANCAS/MP,COORDENACAO-GERAL DE ORC. FIN.E CONTABILIDADE,COORDENACAO-GERAL DE ORCAM.FINAN.CONTAB/MDIC,COORDENACAO-GERAL DE ORCAMENTO E FINANCAS,COORDENACAO-GERAL DE ORCAMENTO E FINANCAS-MJ,COORDENACAO-GERAL DE ORCAMENTO E FINANCAS-MRE,COORDENACAO-GERAL DE RECURSOS LOGISTICOS,COORDENACAO-GERAL DO FUNDO DE TERRAS,CREDITO FUNDIARIO,D CONT - SETORIAL FINANCEIRA,DEPART.NAC.INFRA ESTRUTURA TRANSPORTES.,DEPARTAMENTO DE GESTAO INTERNA,"DEPARTAMENTO DE PLANEJ, ORC E FINANCAS (MD)",DEPARTAMENTO NAC. DE OBRAS CONTRA AS SECAS/UO,DEPARTAMENTO NAC. DE PRODUCAO MINERAL - SEDE,DEPARTAMENTO NAC. ESTRADAS E RODAGEM-EXTINCAO,DEPEN - DIRETORIA EXECUTIVA,DEPTO.DE POLICIA RODOVIARIA FEDERAL,DIRETORIA DE ECON E FINANCAS DA AERONAUTICA,DIRETORIA DE FINANCAS DA MARINHA - AMAZUL,DIRETORIA DE FINANCAS DA MARINHA - CCCPM,DIRETORIA DE FINANCAS DA MARINHA-SISTEMA-PAIS,DIRETORIA DE FINANCAS-SISTEMA PARA O PAIS/MM,DIRETORIA DE GESTAO INTERNA(CGU),DIRETORIA DE GESTAO INTERNA/SE/CGU,DPTO. DE ADMINISTRACAO FINANCEIRA-DAF - INCRA,EMBRAPA-SETORIAL,EMPRESA BRASIL DE COMUNICACAO S.A,EMPRESA BRASILEIRA DE CORREIOS E TELEGRAFOS,EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES,EMPRESA DE PESQUISA ENERGETICA,EMPRESA DE PLANEJAMENTO E LOGISTICA S.A - EPL,EMPRESA DE TRENS URBANOS DE PORTO ALEGRE S/A,FDO EXERCITO - DIFERENCA CAMBIAL,FINANCIADORA DE ESTUDOS E PROJETOS,FUND. UNIVERSIDADE FEDERAL VALE SAO FRANCISCO,FUND.COORD.DE APERF.DE PESSOAL NIVEL SUPERIOR,FUND.UNIV.FED.DE.CIENC.DA SAUDE DE P.ALEGRE,FUNDACAO ALEXANDRE DE GUSMAO,FUNDACAO BIBLIOTECA NACIONAL,FUNDACAO CASA DE RUI BARBOSA,FUNDACAO CULTURAL PALMARES,FUNDACAO ESCOLA NACIONAL DE ADM. PUBLICA,FUNDACAO IBGE-ADMINISTRACAO CENTRAL/RJ,FUNDACAO JOAQUIM NABUCO,FUNDACAO NACIONAL DE ARTES,FUNDACAO NACIONAL DE SAUDE - AP,FUNDACAO NACIONAL DE SAUDE - DF,FUND

We can also group them by hierarchies:

In [17]:
df.groupby(['Superior Agency', 'Agency', 'Managing Unit'])['Actual Amount'].describe().T

Superior Agency                               Advocacia-Geral da União  \
Agency          Advocacia-Geral da União - Unidades com vínculo direto   
Managing Unit            COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU   
count                                                 3.728000e+03       
mean                                                  7.514325e+04       
std                                                   1.497437e+06       
min                                                  -2.692618e+07       
25%                                                   1.630900e+02       
50%                                                   1.624320e+03       
75%                                                   1.035065e+04       
max                                                   6.382985e+07       

Superior Agency       Controladoria-Geral da União  \
Agency                Controladoria-Geral da União   
Managing Unit   DIRETORIA DE GESTAO INTERNA/SE/CGU   
count                                   158.000000   
mean                                  26935.733038   
std                                   93268.862455   
min                                       2.000000   
25%                                      60.287500   
50%                                     766.480000   
75%                                    5659.000000   
max                                  657603.790000   

Superior Agency                                                             \
Agency          Controladoria-Geral da União - Unidades com vínculo direto   
Managing Unit                             DIRETORIA DE GESTAO INTERNA(CGU)   
count                                                 7.500000e+01           
mean                                                  4.220010e+04           
std                                                   1.753453e+05           
min                                                   6.000000e+00           
25%                                                   4.117650e+02           
50%                                                   1.965700e+03           
75%                                                   1.752325e+04           
max                                                   1.410733e+06           

Superior Agency Ministério da Agricultura, Pecuária e Abastec  \
Agency                    Companhia Nacional de Abastecimento   
Managing Unit             COMPANHIA NACIONAL DE ABASTECIMENTO   
count                                            8.397000e+03   
mean                                             1.210700e+05   
std                                              1.843457e+06   
min                                             -6.539708e+06   
25%                                              1.958600e+02   
50%                                              1.943890e+03   
75%                                              1.402195e+04   
max                                              1.183330e+08   

Superior Agency                                              \
Agency          Empresa Brasileira de Pesquisa Agropecuária   
Managing Unit                              EMBRAPA-SETORIAL   
count                                          1.289400e+04   
mean                                           3.051815e+04   
std                                            3.008823e+05   
min                                           -4.667153e+06   
25%                                            1.345100e+02   
50%                                            1.536305e+03   
75%                                            8.240055e+03   
max                                            1.488925e+07   

Superior Agency                                               \
Agency                  Fundo de Defesa da Economia Cafeeira   
Managing Unit   FUNDO DE DEFESA DA ECON.CAFEEIRA/FUNCAFE/SPA   
count                                           3.869000e+03   
mean                                            8.816051e+06   
s